# Numba

Pieces of this lecture were taken from the [QuantEcon lecture on Numba](https://python-programming.quantecon.org/numba.html).

If you'd like to know more about these tools, we recommend reading the QuantEcon lecture and the (very good) [Numba documentation](https://numba.readthedocs.io/en/stable/index.html#)

In [ ]:
import numba
import numpy as np
import pandas as pd

## Compiled vs Interpreted

You may have heard about the differences between "compiled programming languages" and "interpreted programming languages"

* A compiled language is run in a few steps:
  1. Programmer writes the code
  2. Compiler converts that code into machine code
  3. Computer runs machine code. Note that once the code is compiled, it can be run whenever one wants without the compilation step
* An interpreted language runs code differently:
  1. Programmer writes code
  2. Computer "runs" the code by
    * An "interpreter" reads the code line-by-line
    * For each line, the interpreter figures out what the inputs are and tries to convert it to machine code
    * Computer runs the machine code

**Pros and cons of compiled**

* Once the compiler has run, the code is already machine code and runs very fast (as fast as possible given the code you wrote)
* For very large programs, compilation requires the upfront cost of compilation which can take minutes/hours
* Compiled programs can only be shared within similar hardware architecture and operating systems (though as long as there's a compiler for the hardware/OS, one could recompile the code)

**Pros and cons of interpreted**

* As long as there is an interpreter for the hardware/operating system, interpreted code can be easily shared
* Significantly slower than compiled code because of the back and forth to read the code line-by-line (which has to be redone each time the code is run!)
* Easier to interact with your code (and more importantly, your data!) because you can run one line at a time

## Just-in-time compiled (JIT)

JIT is a relatively modern development which has the goal of bridging some of the gaps between compiled and interpreted.

Rather than compile the code ahead of time or interpreting line-by-line, JIT compiles small chunks of the code right before it runs them.

For example, imagine that we have a function `mc_approximate_pi` that approximates the value of pi using Monte-carlo methods... We might even want to run this function multiple times to average across the approximations. The way JIT works is,

1. Check the input types to the function
2. The first time it sees particular types of inputs to the function, it compiles the function assuming those types as inputs and stores this compiled code
3. The computer then runs the function using the compiled code -- If it has seen these inputs before, it can jump directly to this step.

### Our favorite JIT tools

* `Numba`: [Numba](https://numba.pydata.org/) is a package built for Python that adds JIT compilation capabilities for a subset of the Python programming languages -- The priority has been tools for scientific computing `numpy` etc... The main drawback is that only certain packages work with JIT.
* `Julia`: [Julia](https://julialang.org/) is an exciting new language that is based entirely around JIT compilation. The fact that the language is built around JIT means that all packages interact nicely with one another while maintaining their JIT capabilities.

### Numba

As mentioned, Numba is a Python package that adds JIT compilation to a subset of the language using the LLVM compiler library


What works within Numba?

* Many Python objects. including: lists, tuples, dictionaries, integers, floats, strings
* Python logic, including: `if.. elif.. else`, `while`, `for .. in`, `break`, `continue`
* NumPy arrays
* Many (but not all!) NumPy functions

For more information, read these sections from the documentation

* [Supported Python features](https://numba.readthedocs.io/en/stable/reference/pysupported.html)
* [Supported NumPy  features](https://numba.readthedocs.io/en/stable/reference/numpysupported.html)

When to use Numba?

* Loops!!!
* Can facilitate parallelization (we won't talk about this today)
* GPU code generation (we won't talk about this today)
* Did we say loops yet?

### Example

Let's begin with the example we described above by writing a function that approximates pi.

In [ ]:
def calculate_pi(n=1_000_000):
    """
    Approximates pi by drawing two random numbers and
    determining whether the of the sum of their squares
    is less than one (which tells us if the points are
    in the upper-right quadrant of the unit circle). The
    fraction of draws in the upper-quadrant approximates
    the area which we can then multiply by 4 to get the
    area of the circle (which is pi since r=1)
    """
    in_circ = 0

    # Iterate for many samples
    for i in range(n):
        # Draw random numbers
        x = np.random.random()
        y = np.random.random()

        if (x**2 + y**2) < 1:
            in_circ += 1

    return 4 * (in_circ / n)


In [ ]:
%%time

# Vanilla Python function
calculate_pi(5_000_000)

In [ ]:
%%time

# JIT function
calculate_pi_numba = numba.jit(calculate_pi)
calculate_pi_numba(5_000_000)

In [ ]:
%%time

calculate_pi_numba(5_000_000)

Why was the second run faster?

Remember the order than JIT works -- The first time it sees a particular function with given inputs, it has to compile the function

### Pandas?

Does Numba work with Pandas?

In [ ]:
def fill_dataframe(n):
    # Create empty dataframe
    df = pd.DataFrame(index=np.arange(n), columns=["A", "B"])

    a0, b0 = 0.1, 0.2
    for i in range(n):
        df.at[i, "A"] = a0
        df.at[i, "B"] = b0

        # Chaotic
        a0 = 4 * a0 * (1 - a0)
        b0 = 4 * b0 * (1 - b0)

    return df

In [ ]:
fill_dataframe(100)

In [ ]:
fill_dataframe_numba = numba.jit(fill_dataframe)

fill_dataframe_numba(100)

**Object mode vs no Python mode**

* Object mode: Allows Numba to call out to the Python interpreter if it sees something that it doesn't recognize - The cost is that this is slow and requires Numba to make certain optimization sacrifices
* No Python mode: If it sees an object that Numba doesn't recognize, it throws an error. This helps allow Numba make additional optimizations.

Numba's default behavior used to be to compile things in "object" mode but, recently, they've decided to reverse the default behavior to be no Python mode because it was the main use case (and how they recommend people use it).

**Alternate way to fill the DataFrame**


In [ ]:
@numba.jit(nopython=True)
def simulate_chaos(n):
    a = np.zeros(n)
    b = np.zeros(n)
    
    a0, b0 = 0.1, 0.2
    for i in range(n):
        a[i] = a0
        b[i] = b0

        a0 = 4 * a0 * (1 - a0)
        b0 = 4 * b0 * (1 - b0)
    
    return a, b


def fill_dataframe2(n):
    df = pd.DataFrame(index=np.arange(n), columns=["A", "B"])

    a, b = simulate_chaos(n)
    df.loc[:, "A"] = a
    df.loc[:, "B"] = b

    return df

In [ ]:
%%time

fill_dataframe(100_000)

In [ ]:
%%time

fill_dataframe_numba(100_000)

In [ ]:
%%time

fill_dataframe2(100_000)